In [1]:
import sys
sys.path.append('/opt/conda/lib/python3.12/site-packages')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")

import gzip, json
from Bio import SeqIO
import scipy as sp
from collections import Counter
import numpy as np
import pandas as pd
import pickle, os, gzip, json, sys, itertools
from pathlib import Path
from importlib import reload
from dataclasses import dataclass, field
import collections
import matplotlib.pyplot as plt
import networkx as nx
import  sklearn

In [2]:
sys.path.append("scripts")
sys.path.append("../../scripts")
from data_io import is_fwd_id, get_fwd_id, get_sibling_id
from dim_reduction import SpectralEmbedding, scBiMapEmbedding
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges
from truth import get_overlaps
from evaluate import NearestNeighborsConfig, mp_compute_nearest_neighbors
from plots import plot_read_graph, mp_plot_read_graphs, get_graphviz_layout, get_umap_layout
from nearest_neighbors import (
    ExactNearestNeighbors,
    NNDescent,
    WeightedLowHash,
    PAFNearestNeighbors,
    LowHash,
    HNSW,
    ProductQuantization,
    _NearestNeighbors,
    IVFProductQuantization,
)

/home/miaocj/docker_dir/kNN-overlap-finder/scripts/../lib


In [5]:
MAX_SAMPLE_SIZE = int(1e9)
COVERAGE_DEPTH = 20
max_n_neighbors = 20
npz_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/chr22/ONT_R9/kmer_k16/feature_matrix.npz"
tsv_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/chr22/ONT_R9//kmer_k16/metadata.tsv.gz"
json_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/chr22/ONT_R9/kmer_k16/read_features.json.gz"

meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

In [5]:
feature_matrix.shape

(139854, 20092232)

In [6]:
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

In [ ]:
##dimension reduction TruncatedSVD
from sklearn.decomposition import TruncatedSVD 
tsvd = TruncatedSVD(n_components=500)
Trun_dim_mat = tsvd.fit(feature_matrix).transform(feature_matrix)

print(Trun_dim_mat.shape)


(5106, 500)


In [ ]:
##dimension reduction pca 
from sklearn.decomposition import PCA
pca = PCA(n_components=500)  # Reduce to 100 dimensions
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [ ]:
##dimension reduction LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=9)  # Reduce to 9 dimensions
X_train_lda = lda.fit_transform(X_train_scaled, y_train)
X_test_lda = lda.transform(X_test_scaled)

In [29]:
config1 = NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='ExactNearestNeighbors,Euclidean,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='euclidean'))
config2 = NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='ExactNearestNeighbors,Cosine,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='cosine'))
processes = 8
configs = [config2]

nbr_dict, time_dict, memory_dict = mp_compute_nearest_neighbors(
    data=feature_matrix,
    configs=configs,
    n_neighbors=max_n_neighbors,
    processes=processes,
)
nbr_indices = nbr_dict[0]
df_rows = []
for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'Exact_Euclidean', "n_neighbors": k,
                **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

Evaluating 1 configs using 8 processes.
0 Nearest neighbors.
Finished NearestNeighborsConfig(description='ExactNearestNeighbors,Cosine,None,None', tfidf='None', dimension_reduction_method=None, nearest_neighbors_method=<class 'nearest_neighbors.ExactNearestNeighbors'>). Elapsed time: {'nearest_neighbors': 4.065868854522705}. Peak memory: {}



In [45]:
processes = 8
read_ids = np.array(list(read_features))
graphs = collections.defaultdict(dict)
k_values = np.arange(2, max_n_neighbors + 1)
def from_matrix_to_evaluation(feature_matrix):
    config = NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,TF',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 

    configs = [config]
    nbr_dict, time_dict, memory_dict = mp_compute_nearest_neighbors(
        data=feature_matrix,
        configs=configs,
        n_neighbors=max_n_neighbors,
        processes=processes,
    )
    nbr_indices = nbr_dict[0]
    df_rows = []
    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'Exact_Euclidean', "n_neighbors": k,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    return df, nbr_indices



In [44]:
##Exact eud scbimap 500d
df,nbr_indices = from_matrix_to_evaluation(feature_matrix)
df

Evaluating 1 configs using 8 processes.
0 Dimension reduction.
Nearest neighbors.
Finished NearestNeighborsConfig(description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,TF', tfidf='None', dimension_reduction_method=<class 'dim_reduction.scBiMapEmbedding'>, nearest_neighbors_method=<class 'nearest_neighbors.ExactNearestNeighbors'>). Elapsed time: {'dimension_reduction': 97.01570558547974, 'nearest_neighbors': 1.103588342666626}. Peak memory: {}



,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,Exact_Euclidean,2,0.810893,0.325131,0.056805,0.238394,1052,0.206032,21,0.41%
1,Exact_Euclidean,3,0.800568,0.283633,0.081944,0.303872,616,0.120642,87,1.70%
2,Exact_Euclidean,4,0.793278,0.259233,0.106502,0.364281,372,0.072855,87,1.70%
3,Exact_Euclidean,5,0.784826,0.237409,0.130227,0.412325,220,0.043087,166,3.25%
4,Exact_Euclidean,6,0.777881,0.218730,0.153390,0.451449,142,0.027810,383,7.50%
5,Exact_Euclidean,7,0.773061,0.207897,0.176515,0.496858,94,0.018410,387,7.58%
6,Exact_Euclidean,8,0.768157,0.196557,0.199214,0.533550,64,0.012534,387,7.58%
7,Exact_Euclidean,9,0.763069,0.187861,0.221215,0.570039,38,0.007442,440,8.62%
8,Exact_Euclidean,10,0.757785,0.179902,0.242945,0.603689,34,0.006659,446,8.73%
9,Exact_Euclidean,11,0.752637,0.171719,0.263998,0.630448,28,0.005484,448,8.77%


In [ ]:
##Exact eud spectrual 500d
df,nbr_indices = from_matrix_to_evaluation(feature_matrix)
df

In [ ]:
processes = 8
read_ids = np.array(list(read_features))
graphs = collections.defaultdict(dict)
k_values = np.arange(2, max_n_neighbors + 1)
def from_matrix_to_evaluation(feature_matrix):
    config = NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,TF',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 

    configs = [config]
    nbr_dict, time_dict, memory_dict = mp_compute_nearest_neighbors(
        data=feature_matrix,
        configs=configs,
        n_neighbors=max_n_neighbors,
        processes=processes,
    )
    nbr_indices = nbr_dict[0]
    df_rows = []
    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'Exact_Euclidean', "n_neighbors": k,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    return df, nbr_indices

In [ ]:
df,nbr_indices = from_matrix_to_evaluation(feature_matrix)
df

Evaluating 1 configs using 8 processes.
0 Nearest neighbors.
Finished NearestNeighborsConfig(description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,TF', tfidf='None', dimension_reduction_method=None, nearest_neighbors_method=<class 'nearest_neighbors.ExactNearestNeighbors'>). Elapsed time: {'nearest_neighbors': 6.122572422027588}. Peak memory: {}



,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,Exact_Euclidean,2,0.671766,0.250919,0.054858,0.214474,1380,0.270270,17,0.33%
1,Exact_Euclidean,3,0.644535,0.222594,0.078399,0.283398,896,0.175480,47,0.92%
2,Exact_Euclidean,4,0.615938,0.197424,0.099549,0.333975,612,0.119859,116,2.27%
3,Exact_Euclidean,5,0.591209,0.179307,0.119575,0.379586,457,0.089503,118,2.31%
4,Exact_Euclidean,6,0.567460,0.163422,0.137461,0.414352,374,0.073247,122,2.39%
5,Exact_Euclidean,7,0.547518,0.152976,0.154649,0.452260,305,0.059734,202,3.96%
6,Exact_Euclidean,8,0.528266,0.141946,0.170753,0.480235,260,0.050920,216,4.23%
7,Exact_Euclidean,9,0.509678,0.133082,0.185879,0.508007,224,0.043870,478,9.36%
8,Exact_Euclidean,10,0.493899,0.124738,0.200686,0.530509,198,0.038778,478,9.36%
9,Exact_Euclidean,11,0.478238,0.117798,0.214301,0.552504,174,0.034078,478,9.36%


In [ ]:
#SpectralEmbedding
def from_matrix_to_evaluation(feature_matrix):
    config = NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,500,TF',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 

    configs = [config]
    nbr_dict, time_dict, memory_dict = mp_compute_nearest_neighbors(
        data=feature_matrix,
        configs=configs,
        n_neighbors=max_n_neighbors,
        processes=processes,
    )
    nbr_indices = nbr_dict[0]
    df_rows = []
    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'Exact_Euclidean', "n_neighbors": k,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    return df, nbr_indices

In [38]:
df,nbr_indices = from_matrix_to_evaluation(feature_matrix)
df

Evaluating 1 configs using 8 processes.
0 Dimension reduction.
Nearest neighbors.
Finished NearestNeighborsConfig(description='HNSW,cosine,scBiMapEmbedding,500,None', tfidf='None', dimension_reduction_method=<class 'dim_reduction.scBiMapEmbedding'>, nearest_neighbors_method=<class 'nearest_neighbors.HNSW'>). Elapsed time: {'dimension_reduction': 86.1794981956482, 'nearest_neighbors': 0.10550355911254883}. Peak memory: {}



,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,Exact_Euclidean,2,0.811446,0.325408,0.056843,0.238597,1050,0.205640,21,0.41%
1,Exact_Euclidean,3,0.800757,0.283822,0.081963,0.304075,616,0.120642,87,1.70%
2,Exact_Euclidean,4,0.793710,0.259233,0.106560,0.364281,372,0.072855,87,1.70%
3,Exact_Euclidean,5,0.785352,0.237409,0.130314,0.412325,219,0.042891,166,3.25%
4,Exact_Euclidean,6,0.778471,0.218828,0.153506,0.451652,138,0.027027,383,7.50%
5,Exact_Euclidean,7,0.773527,0.207982,0.176622,0.497061,92,0.018018,387,7.58%
6,Exact_Euclidean,8,0.768493,0.196632,0.199301,0.533752,62,0.012143,387,7.58%
7,Exact_Euclidean,9,0.763403,0.188062,0.221312,0.570647,39,0.007638,439,8.60%
8,Exact_Euclidean,10,0.758027,0.179992,0.243023,0.603994,35,0.006855,440,8.62%
9,Exact_Euclidean,11,0.752885,0.171746,0.264085,0.630549,27,0.005288,448,8.77%


In [9]:
processes = 8
read_ids = np.array(list(read_features))
graphs = collections.defaultdict(dict)
k_values = np.arange(2, max_n_neighbors + 1)
def from_nbr_to_evaluation(nbr_indices):
    df_rows = []
    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'Exact_Euclidean', "n_neighbors": k,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    return df

In [17]:
nbrs = sklearn.neighbors.NearestNeighbors(
    n_neighbors=20,metric='euclidean'
)
nbrs.fit(feature_matrix)
_, nbr_indices2 = nbrs.kneighbors(feature_matrix)
df = from_nbr_to_evaluation(nbr_indices2)
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,Exact_Euclidean,2,0.671766,0.250919,0.054858,0.214474,1380,0.270270,17,0.33%
1,Exact_Euclidean,3,0.644535,0.222594,0.078399,0.283398,896,0.175480,47,0.92%
2,Exact_Euclidean,4,0.615938,0.197424,0.099549,0.333975,612,0.119859,116,2.27%
3,Exact_Euclidean,5,0.591209,0.179307,0.119575,0.379586,457,0.089503,118,2.31%
4,Exact_Euclidean,6,0.567460,0.163422,0.137461,0.414352,374,0.073247,122,2.39%
5,Exact_Euclidean,7,0.547518,0.152976,0.154649,0.452260,305,0.059734,202,3.96%
6,Exact_Euclidean,8,0.528266,0.141946,0.170753,0.480235,260,0.050920,216,4.23%
7,Exact_Euclidean,9,0.509678,0.133082,0.185879,0.508007,224,0.043870,478,9.36%
8,Exact_Euclidean,10,0.493899,0.124738,0.200686,0.530509,198,0.038778,478,9.36%
9,Exact_Euclidean,11,0.478238,0.117798,0.214301,0.552504,174,0.034078,478,9.36%


In [18]:
nbrs = sklearn.neighbors.NearestNeighbors(
    n_neighbors=20,metric='cosine'
)
nbrs.fit(feature_matrix)
_, nbr_indices2 = nbrs.kneighbors(feature_matrix)
df = from_nbr_to_evaluation(nbr_indices2)
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,Exact_Euclidean,2,0.835956,0.305183,0.069975,0.267383,888,0.173913,47,0.92%
1,Exact_Euclidean,3,0.823697,0.272354,0.100982,0.349483,488,0.095574,140,2.74%
2,Exact_Euclidean,4,0.812056,0.247263,0.130711,0.416582,258,0.050529,156,3.06%
3,Exact_Euclidean,5,0.801250,0.226572,0.158968,0.470505,178,0.034861,202,3.96%
4,Exact_Euclidean,6,0.793493,0.211450,0.186567,0.520373,118,0.023110,396,7.76%
5,Exact_Euclidean,7,0.783672,0.197669,0.212306,0.560511,72,0.014101,396,7.76%
6,Exact_Euclidean,8,0.774610,0.188585,0.236825,0.603487,60,0.011751,400,7.83%
7,Exact_Euclidean,9,0.763605,0.178561,0.260066,0.636529,44,0.008617,474,9.28%
8,Exact_Euclidean,10,0.754808,0.169768,0.282784,0.665721,36,0.007051,474,9.28%
9,Exact_Euclidean,11,0.745662,0.162700,0.304631,0.695723,26,0.005092,484,9.48%


In [25]:
nbrs = sklearn.neighbors.NearestNeighbors(
    n_neighbors=20,metric='cosine',algorithm='brute'
)
nbrs.fit(feature_matrix)
_, nbr_indices2 = nbrs.kneighbors(feature_matrix)
df = from_nbr_to_evaluation(nbr_indices2)
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,Exact_Euclidean,2,0.835956,0.305183,0.069975,0.267383,888,0.173913,47,0.92%
1,Exact_Euclidean,3,0.823697,0.272354,0.100982,0.349483,488,0.095574,140,2.74%
2,Exact_Euclidean,4,0.812056,0.247263,0.130711,0.416582,258,0.050529,156,3.06%
3,Exact_Euclidean,5,0.801250,0.226572,0.158968,0.470505,178,0.034861,202,3.96%
4,Exact_Euclidean,6,0.793493,0.211450,0.186567,0.520373,118,0.023110,396,7.76%
5,Exact_Euclidean,7,0.783672,0.197669,0.212306,0.560511,72,0.014101,396,7.76%
6,Exact_Euclidean,8,0.774610,0.188585,0.236825,0.603487,60,0.011751,400,7.83%
7,Exact_Euclidean,9,0.763605,0.178561,0.260066,0.636529,44,0.008617,474,9.28%
8,Exact_Euclidean,10,0.754808,0.169768,0.282784,0.665721,36,0.007051,474,9.28%
9,Exact_Euclidean,11,0.745662,0.162700,0.304631,0.695723,26,0.005092,484,9.48%


In [53]:
mydim = scBiMapEmbedding()
new_matrix = mydim.transform(feature_matrix,n_dimensions=500)

In [54]:
nbrs = sklearn.neighbors.NearestNeighbors(
    n_neighbors=20,metric='euclidean',algorithm='auto'
)
nbrs.fit(new_matrix)
_, nbr_indices2 = nbrs.kneighbors(new_matrix)
df = from_nbr_to_evaluation(nbr_indices2)
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,Exact_Euclidean,2,0.810893,0.325131,0.056805,0.238394,1052,0.206032,21,0.41%
1,Exact_Euclidean,3,0.800568,0.283633,0.081944,0.303872,616,0.120642,87,1.70%
2,Exact_Euclidean,4,0.793278,0.259233,0.106502,0.364281,372,0.072855,87,1.70%
3,Exact_Euclidean,5,0.784826,0.237409,0.130227,0.412325,220,0.043087,166,3.25%
4,Exact_Euclidean,6,0.777881,0.218730,0.153390,0.451449,142,0.027810,383,7.50%
5,Exact_Euclidean,7,0.773061,0.207897,0.176515,0.496858,94,0.018410,387,7.58%
6,Exact_Euclidean,8,0.768157,0.196557,0.199214,0.533550,64,0.012534,387,7.58%
7,Exact_Euclidean,9,0.763069,0.187861,0.221215,0.570039,38,0.007442,440,8.62%
8,Exact_Euclidean,10,0.757785,0.179902,0.242945,0.603689,34,0.006659,446,8.73%
9,Exact_Euclidean,11,0.752637,0.171719,0.263998,0.630448,28,0.005484,448,8.77%


In [50]:
nbrs = sklearn.neighbors.NearestNeighbors(
    n_neighbors=20,metric='euclidean',algorithm='auto'
)
nbrs.fit(feature_matrix)
_, nbr_indices2 = nbrs.kneighbors(feature_matrix)
df = from_nbr_to_evaluation(nbr_indices2)
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,Exact_Euclidean,2,0.671766,0.250919,0.054858,0.214474,1380,0.270270,17,0.33%
1,Exact_Euclidean,3,0.644535,0.222594,0.078399,0.283398,896,0.175480,47,0.92%
2,Exact_Euclidean,4,0.615938,0.197424,0.099549,0.333975,612,0.119859,116,2.27%
3,Exact_Euclidean,5,0.591209,0.179307,0.119575,0.379586,457,0.089503,118,2.31%
4,Exact_Euclidean,6,0.567460,0.163422,0.137461,0.414352,374,0.073247,122,2.39%
5,Exact_Euclidean,7,0.547518,0.152976,0.154649,0.452260,305,0.059734,202,3.96%
6,Exact_Euclidean,8,0.528266,0.141946,0.170753,0.480235,260,0.050920,216,4.23%
7,Exact_Euclidean,9,0.509678,0.133082,0.185879,0.508007,224,0.043870,478,9.36%
8,Exact_Euclidean,10,0.493899,0.124738,0.200686,0.530509,198,0.038778,478,9.36%
9,Exact_Euclidean,11,0.478238,0.117798,0.214301,0.552504,174,0.034078,478,9.36%


In [ ]:
nbrs = sklearn.neighbors.NearestNeighbors(
    n_neighbors=20,metric='cosine',algorithm='brute'
)
nbrs.fit(feature_matrix)
_, nbr_indices2 = nbrs.kneighbors(feature_matrix)
df = from_nbr_to_evaluation(nbr_indices2)
df

In [48]:
with open('/home/miaocj/docker_dir/kNN-overlap-finder/workflow/notebooks/config_dict.pkl', 'rb') as file:  
    config_dict = pickle.load(file)

In [49]:
config_dict['Exact_Euclidean_None_None']

NearestNeighborsConfig(description='ExactNearestNeighbors,Euclidean,None,None', tfidf='None', dimension_reduction_method=None, nearest_neighbors_method=<class 'nearest_neighbors.ExactNearestNeighbors'>)